In [7]:
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("DS_KEY")
api_base = os.getenv("DS_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base

In [ ]:
from langchain.chains import load_chain
chain = load_chain("lc:/chains/chain.json")
print(chain.run("2+6等于几?"))


## CustomChain
当通用链不满足的时候可以自行构建来实现特定的目的。

In [16]:
from typing import List, Any, Optional, Dict

from langchain.callbacks.manager import (CallbackManagerForChainRun)

from langchain.chains.base import Chain
from langchain.prompts.base import BasePromptTemplate
from langchain.base_language import BaseLanguageModel

In [ ]:

class wiki_article_chain(Chain):
    """开发一个wiki文章生成器"""
    prompt:BasePromptTemplate
    llm:BaseLanguageModel
    out_key:str="text"

    @property
    def input_keys(self) -> List[str]:
        """将返回Prompt所需的所有键"""
        return self.prompt.input_variables
    
    @property
    def output_keys(self) -> List[str]:
        """将始终返回text键"""
        return [self.out_key]
    
    def _call(
        self,
        inputs:Dict[str,Any],
        run_manager:Optional[CallbackManagerForChainRun]=None,
    ) -> Dict[str,Any]:
        """运行链"""
        prompt_value = self.prompt.format_prompt(**inputs)
        #print("prompt_value:",prompt_value)
        response = self.llm.generate_prompt(
            [prompt_value],callbacks=run_manager.get_child() if run_manager else None
        )
        #print("response:",response)
        if run_manager:
            run_manager.on_text("wiki article is written")
        return {self.out_key:response.generations[0][0].text}
    
    @property
    def _chain_type(self) -> str:
        """链类型"""
        return "wiki_article_chain"

In [30]:

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

print(os.getenv("DS_KEY"))
print(os.getenv("DS_API_BASE"))
chain = wiki_article_chain(
    baseMode= None,
    prompt=PromptTemplate(template="Create a wiki article about {topic}"),
    llm=ChatOpenAI(
        model="deepseek-chat",
        api_key=os.getenv("DS_KEY"),
        base_url=os.getenv("DS_API_BASE"),
        temperature=0)
)

sk-c974b5d99e5f42048860411998b7efa3
https://api.deepseek.com/v1


PydanticUserError: `wiki_article_chain` is not fully defined; you should define `BaseCache`, then call `wiki_article_chain.model_rebuild()`.

For further information visit https://errors.pydantic.dev/2.10/u/class-not-fully-defined

In [20]:
result = chain.run({"topic":"降本增效"})
print(result["text"])

/var/folders/sy/dfvmk27d1hdbrm5h8p4qwm3m0000gn/T/ipykernel_72760/395548382.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain.run({"topic":"降本增效"})


ResponseError: model "deepseek-chat" not found, try pulling it first